# Homework #6

### Используйте датасет с  практики текущего урока.

### Создайте матрицу item-customer (по строкам - товары, по столбцам - покупатели)


In [21]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [9]:
df = pd.read_excel('OnlineRetail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df = df.loc[df['Quantity'] > 0]
df = df.dropna(subset=['CustomerID'])

In [11]:
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [12]:
df['CustomerID'] = df['CustomerID'].astype('int')

In [16]:
item_customer_matrix = df.pivot_table(
    columns='CustomerID',
    index='StockCode', 
    values='Quantity', 
    aggfunc='sum'
)

item_customer_matrix.head()

CustomerID,12346,12347,12348,12349,12350,12352,12353,12354,12355,12356,...,18273,18274,18276,18277,18278,18280,18281,18282,18283,18287
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
item_customer_matrix.shape

(3665, 4339)

In [20]:
item_customer_matrix = item_customer_matrix.applymap(lambda x: 1 if x > 0 else 0)
item_customer_matrix.head()

CustomerID,12346,12347,12348,12349,12350,12352,12353,12354,12355,12356,...,18273,18274,18276,18277,18278,18280,18281,18282,18283,18287
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров.


In [22]:
def cos_sim(array):
  
    csr_array = csr_matrix(array)
    numerator = np.dot(csr_array, csr_array.T).toarray()
    denomenator = np.linalg.norm(array, axis=1, keepdims=True) * np.linalg.norm(array.T, axis=0, keepdims=True)
    return numerator / denomenator

In [24]:
m = item_customer_matrix.values

In [26]:
item_item_sim_matrix = cos_sim(m)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [27]:
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)

item_item_sim_matrix.columns = item_customer_matrix.index

item_item_sim_matrix['CustomerID'] = item_customer_matrix.index
item_item_sim_matrix = item_item_sim_matrix.set_index('CustomerID')

item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


### Отберите ТОП-10 похожих товаров по StockCode.
### Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.
### Исходный товар - StockCode: 23166 Description: MEDIUM CERAMIC TOP STORAGE JAR


In [62]:
to_recommend = item_item_sim_matrix.loc[23166].sort_values(ascending=False).reset_index()
to_recommend['Description'] = df['Description']
to_recommend.head(11)

,StockCode,23166,Description
0,23166,1.000000,WHITE HANGING HEART T-LIGHT HOLDER
1,23165,0.671897,WHITE METAL LANTERN
2,23167,0.659551,CREAM CUPID HEARTS COAT HANGER
3,22993,0.274724,KNITTED UNION FLAG HOT WATER BOTTLE
4,23307,0.262526,RED WOOLLY HOTTIE WHITE HEART.
5,22722,0.249053,SET 7 BABUSHKA NESTING BOXES
6,22720,0.249002,GLASS STAR FROSTED T-LIGHT HOLDER
7,22666,0.247110,HAND WARMER UNION JACK
8,23243,0.246397,HAND WARMER RED POLKA DOT
9,22961,0.242447,ASSORTED COLOUR BIRD ORNAMENT
